In [5]:
import pandas as pd
import numpy as np
from pprint import pprint
import os

from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.algorithms.postprocessing import RejectOptionClassification
from aif360.detectors.mdss.ScoringFunctions import Bernoulli
from aif360.detectors.mdss.MDSS import MDSS
from aif360.detectors.mdss.generator import get_random_subset

from IPython.display import Markdown, display

In [7]:
ad_conversion_dataset = pd.read_csv(os.path.join('..','data','ad_campaign_data.csv'))
ad_conversion_dataset.head()

,religion,politics,college_educated,parents,homeowner,gender,age,income,area,true_conversion,predicted_conversion,predicted_probability
0,Unknown,Unknown,1,1,1,Unknown,55-64,Unknown,Unknown,0,0,0.001351
1,Other,Unknown,1,1,1,Unknown,55-64,Unknown,Urban,0,0,0.002238
2,Unknown,Unknown,1,1,1,F,55-64,Unknown,Unknown,0,0,0.002704
3,Unknown,Unknown,1,1,1,F,55-64,Unknown,Unknown,0,0,0.001967
4,Unknown,Unknown,1,1,1,F,55-64,Unknown,Urban,0,0,0.001681


In [8]:
features_4_scanning = ['college_educated','parents','homeowner','gender','age','income','area','politics','religion']

In [9]:
def print_report(data, subset):
    """ Utility function to pretty-print the subsets."""
    if subset:
        to_choose = ad_conversion_dataset[subset.keys()].isin(subset).all(axis = 1)
        df = ad_conversion_dataset[['true_conversion', 'predicted_conversion']][to_choose]
    else:
        for col in features_4_scanning:
            subset[col] = list(ad_conversion_dataset[col].unique())
        df = ad_conversion_dataset[['true_conversion', 'predicted_conversion']]

    true = df['true_conversion'].sum()
    pred = df['predicted_conversion'].sum()
    
    print('\033[1mSubset: \033[0m')
    pprint(subset)
    print('\033[1mSubset Size: \033[0m', len(df))
    print('\033[1mTrue Clicks: \033[0m', true)
    print('\033[1mPredicted Clicks: \033[0m', pred)
    print()

In [10]:
np.random.seed(11)
random_subset = get_random_subset(ad_conversion_dataset[features_4_scanning], prob = 0.05, min_elements = 10000)
print_report(ad_conversion_dataset, random_subset)

Subset: 
{'politics': ['Unknown'], 'religion': ['Other']}
Subset Size:  214101
True Clicks:  357
Predicted Clicks:  376



In [11]:
# Bias scan
scoring_function = Bernoulli(direction='negative')
scanner = MDSS(scoring_function)
 
scanned_subset, _ = scanner.scan(ad_conversion_dataset[features_4_scanning], 
                        expectations = ad_conversion_dataset['predicted_conversion'],
                        outcomes = ad_conversion_dataset['true_conversion'], 
                        penalty = 1, 
                        num_iters = 1,
                        verbose = False)

print_report(ad_conversion_dataset, scanned_subset)

Subset: 
{'area': ['Unknown', 'Urban'], 'homeowner': [0], 'income': ['>100K', 'Unknown']}
Subset Size:  153883
True Clicks:  281
Predicted Clicks:  1907



In [12]:
def convert_to_standard_dataset(df, target_label_name, scores_name=""):

    # List of names corresponding to protected attribute columns in the dataset.
    # Note that the terminology "protected attribute" used in AI Fairness 360 to
    # divide the dataset into multiple groups for measuring and mitigating 
    # group-level bias.
    protected_attributes=['homeowner']
    
    # columns from the dataset that we want to select for this Bias study
    selected_features = ['gender', 'age', 'income', 'area', 'college_educated', 'homeowner', 
                         'parents', 'predicted_probability']
    
    # This privileged class is selected based on MDSS subgroup evaluation.
    # in previous steps. In our case non-homeowner (homeowner=0) are considered to 
    # be privileged and homeowners (homeowner=1) are considered as unprivileged.
    privileged_classes = [[0]]   

    # Label values which are considered favorable are listed. All others are 
    # unfavorable. Label values are mapped to 1 (favorable) and 0 (unfavorable) 
    # if they are not already binary and numerical.
    favorable_target_label = [1]

    # List of column names in the DataFrame which are to be expanded into one-hot vectors.
    categorical_features = ['parents','gender','college_educated','area','income', 'age']

    # create the `StandardDataset` object
    standard_dataset = StandardDataset(df=df, label_name=target_label_name,
                                    favorable_classes=favorable_target_label,
                                    scores_name=scores_name,
                                    protected_attribute_names=protected_attributes,
                                    privileged_classes=privileged_classes,
                                    categorical_features=categorical_features,
                                    features_to_keep=selected_features)
    if scores_name=="":
        standard_dataset.scores = standard_dataset.labels.copy()
        
    return standard_dataset

In [13]:
# Create two StandardDataset objects - one with true conversions and one with
# predicted conversions.

# First create the predicted dataset
ad_standard_dataset_pred = convert_to_standard_dataset(ad_conversion_dataset, 
                                            target_label_name = 'predicted_conversion',
                                            scores_name='predicted_probability')

# Use this to create the original dataset
ad_standard_dataset_orig = ad_standard_dataset_pred.copy()
ad_standard_dataset_orig.labels = ad_conversion_dataset["true_conversion"].values.reshape(-1, 1)
ad_standard_dataset_orig.scores = ad_conversion_dataset["true_conversion"].values.reshape(-1, 1)